#Lista 3
-------------


Objetivo: agrupar conjunto de observações de um banco de dados.

Banco utilizado: Wholesale customer

Descrição do dados:

1. FRESH: Gastos anuais (m.u.) em produtos frescos (Contínuo)
2. MILK: Gastos anuais (m.u.) em produtos lácteos (Contínuo)
3. GROCERY: Gastos anuais (m.u.) em produtos de mercearia (Contínuo)
4. FROZEN: Gastos anuais (m.u.) em produtos congelados (Contínuo)
5. DETERGENTS_PAPER: Gastos anuais (m.u.) em detergentes e produtos de papel (Contínuo)
6. DELICATESSEN: Gastos anuais (m.u.) em produtos de delicatessen (Contínuo)
7. CHANNEL: Canal dos clientes - Horeca (Hotel/Restaurante/Café) ou Varejo (Nominal)
8. REGION: Região dos clientes - Lisboa, Porto ou Outras (Nominal)

Exercício:

1. Carregamento e Análise Inicial:
* Carregue o banco de dados e analise suas features.
* Não utilize as variáveis Channel e Region.
* Realize a padronização dos dados.

**Observação**: Nos exercícios seguintes, utilize os dados padronizados.

2. Geração de Clusters:
* Gere 5 clusters utilizando os algoritmos K-means, Aglomerativo e DBSCAN (Utilize eps de 3000 e min_samples de 5).
3. Análise de Silhouette Score:
* Analise o resultado do Silhouette Score para cada agrupamento. Qual dentre os três é o melhor?

4. Decomposição de Componentes Principais (PCA):
* Realize a decomposição de componentes principais (PCA) com duas componentes.
* Qual o percentual da variabilidade é explicado com duas componentes?
* Construa um gráfico de pontos (scatterplot) onde no eixo x temos a primeira componente e no eixo y temos a segunda componente. Colora os pontos com o melhor cluster.

5. Método do Cotovelo (Elbow Method):
* Utilize o método do cotovelo junto com o algoritmo K-means para definir qual o melhor número de grupos.

## 1. Carga do banco
--------------------

In [1]:
import pandas as pd

df = pd.read_csv('../datasource/Wholesale customers data.csv')

In [2]:
df.head()

,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen
0,2,3,12669,9656,7561,214,2674,1338
1,2,3,7057,9810,9568,1762,3293,1776
2,2,3,6353,8808,7684,2405,3516,7844
3,1,3,13265,1196,4221,6404,507,1788
4,2,3,22615,5410,7198,3915,1777,5185


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440 entries, 0 to 439
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   Channel           440 non-null    int64
 1   Region            440 non-null    int64
 2   Fresh             440 non-null    int64
 3   Milk              440 non-null    int64
 4   Grocery           440 non-null    int64
 5   Frozen            440 non-null    int64
 6   Detergents_Paper  440 non-null    int64
 7   Delicassen        440 non-null    int64
dtypes: int64(8)
memory usage: 27.6 KB


In [4]:
df.describe()

,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen
count,440.000000,440.000000,440.000000,440.000000,440.000000,440.000000,440.000000,440.000000
mean,1.322727,2.543182,12000.297727,5796.265909,7951.277273,3071.931818,2881.493182,1524.870455
std,0.468052,0.774272,12647.328865,7380.377175,9503.162829,4854.673333,4767.854448,2820.105937
min,1.000000,1.000000,3.000000,55.000000,3.000000,25.000000,3.000000,3.000000
25%,1.000000,2.000000,3127.750000,1533.000000,2153.000000,742.250000,256.750000,408.250000
50%,1.000000,3.000000,8504.000000,3627.000000,4755.500000,1526.000000,816.500000,965.500000
75%,2.000000,3.000000,16933.750000,7190.250000,10655.750000,3554.250000,3922.000000,1820.250000
max,2.000000,3.000000,112151.000000,73498.000000,92780.000000,60869.000000,40827.000000,47943.000000


In [5]:
df.isna().sum()

Channel             0
Region              0
Fresh               0
Milk                0
Grocery             0
Frozen              0
Detergents_Paper    0
Delicassen          0
dtype: int64

## 2. Geração dos clusters
--------------------
### 2.1. KNN - (K-Nearest Neighbors)
--------------------

In [6]:
# I believe KNN isn't typically used for clustering, as it's primarily a classification and regression algorithm. Is that correct?

# Here, we'll use "channel" as the target variable and treat the remaining columns as features.
X = df.drop('Channel', axis=1) # Features
y = df['Channel']  # Target

In [7]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)


In [8]:
from sklearn.preprocessing import StandardScaler

# Normalizing data with StandardScaler
# KNN is sensitive to differences in feature scales, which can impact its performance.
# To address this, we standardize the data so each feature has a mean of 0 and a standard deviation of 1.
# We'll use StandardScaler from sklearn.preprocessing for this normalization.
scaler = StandardScaler()

# Fit the scaler to the training data
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [9]:
from sklearn.neighbors import KNeighborsClassifier

# Creates a KNN classifier with 3 neighbors
knn = KNeighborsClassifier(n_neighbors=3)

# Train the classifier with a given and standard dataset
knn.fit(X_train, y_train)

# Predict the class labels for the provided data
y_pred = knn.predict(X_test)

In [10]:
from sklearn.metrics import accuracy_score

print(f'Model Accuracy: { accuracy_score(y_test, y_pred)}')

Model Accuracy: 0.8863636363636364


In [21]:
# Use plotly to plot a graph with predicted values and real values. Use a scatter plot.
import plotly.express as px

predicted_df = pd.DataFrame(
    {'real': y_test, 'predicted': y_pred, 'deviation': y_test - y_pred})

fig = px.line(predicted_df, x='real', y='predicted', color='deviation')

fig.show()


In [12]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.97      0.88      0.92        65
           2       0.72      0.91      0.81        23

    accuracy                           0.89        88
   macro avg       0.85      0.89      0.86        88
weighted avg       0.90      0.89      0.89        88



### 2.2. Aglomerativo
--------------------

In [13]:
from sklearn.cluster import AgglomerativeClustering

##2.3. DBSCAN
--------------------

## 3 Análise sillhouete score
---------------------------

## 4. Visualização
---------------------------
## 4.1. PCA
---------------------------

## 4.2. Gráfico
---------------------------

##5. Método elbow
---------------------------